In [1]:
import pandas as pd

#Read the Wteam file
wteamDF = spark.read.csv("/FileStore/tables/WTeams.csv", header=True, inferSchema= True)
lteamDF = spark.read.csv("/FileStore/tables/WTeams.csv", header=True, inferSchema= True)

In [2]:
#Convert into Pandas DF from sql.dataframe
wteamDF = wteamDF.toPandas()
lteamDF = lteamDF.toPandas()
#Rename the column to WTeamName

wteamDF.columns = ['WTeamID', 'WTeamName']
lteamDF.columns = ['LTeamID', 'LTeamName']


In [3]:
#Read the RegularSeasons CSV File
regularSeasonsDF = spark.read.csv("/FileStore/tables/WRegularSeasonCompactResults.csv", header=True, inferSchema= True)


In [4]:
regularSeasonsDF = regularSeasonsDF.toPandas()


In [5]:
lteamDF[:2]

In [6]:
#merge why??
x = regularSeasonsDF.merge(wteamDF, on='WTeamID', how='left')

In [7]:
x[:2]

In [8]:
y=x.merge(lteamDF,on='LTeamID',how='left') 

In [9]:
#To check if all the entries are non-null
y.isnull().sum()

In [10]:
final_regularSeasonsDF = y

In [11]:
final_regularSeasonsDF[:3]

In [12]:
#Merging the Seeds and Slots
seedsDF = spark.read.csv("/FileStore/tables/WNCAATourneySeeds.csv", header=True, inferSchema= True)

slotsDF = spark.read.csv("/FileStore/tables/WNCAATourneySlots.csv", header=True, inferSchema=True)

In [13]:
slotsDF = slotsDF.toPandas()




In [14]:
seedsDF = seedsDF.toPandas()

seedsDF.head(5)

In [15]:
#Concatenate to replicate the rows for 63 matches in each season
slotsDF = pd.concat([slotsDF]*20, ignore_index=True) ###dbt
len(slotsDF['Slot']) #sbt

In [16]:
seedsDF[1279:]

In [17]:
allyears = seedsDF['Season']

slotsDF['Season'] = pd.Series(allyears, dtype='int32')
  

In [18]:
slotsDF[:70]

In [20]:
len(seedsDF['Season'])

In [21]:
slotsDF[1260:]

In [22]:
#for i in range(0, len(seedsDF['Season'])):
  #if(seedsDF['Seed'])

In [23]:
CompactDF = spark.read.csv("/FileStore/tables/WNCAATourneyCompactResults.csv", header=True, inferSchema= True)

NewseedsDF=seedsDF

CompactDF.show()

In [24]:
#DICTIONARY -team id,season as key and seed as value

seedsdict={}
for row in NewseedsDF.iterrows():
  seedsdict[(row[1]["Season"], row[1]["TeamID"])] = row[1]["Seed"]


In [25]:
CompactDF = CompactDF.toPandas()

In [27]:
temp_wCol = []
temp_lCol = []
for row in CompactDF.iterrows():
  year = row[1]['Season']
  wteamid = row[1]['WTeamID']
  lteamid = row[1]['LTeamID']
  temp_wCol.append(seedsdict[(year,wteamid)])
  temp_lCol.append(seedsdict[(year,lteamid)])
  

In [28]:
CompactDF['WSeed'] = temp_wCol
CompactDF['LSeed'] = temp_lCol

CompactDF.head(10)

In [29]:
weights_dict = {}
j = 1
for i in range(16,0,-1):
  weights_dict[j] = i
  j+=1
  print weights_dict  
#temp_val = abs(weights_dict[int(wseed[1:])] - weights_dict[int(lseed[1:])])

In [30]:
temp_win = []
wseed_num = []
lseed_num = []
diff_seed = []
loc_col = []
diff_score = []

for row in CompactDF.iterrows():
  
  team_1 = row[1]['WTeamID']
  team_2 = row[1]['LTeamID']
  loc_val = row[1]['WLoc']
  wseed = row[1]['WSeed']
  lseed =row[1]['LSeed']
#ask why 1 & 0 for temp_win  
  if(team_1<team_2):
    temp_win.append(1)
  else:
    temp_win.append(0)
    
  if(loc_val == 'H'):
    loc_col.append(3)
  elif(loc_val == 'N'):
    loc_col.append(2)
  elif(loc_val=='A'):
    loc_col.append(1)
  
  #print wseed[1:]   
  temp_val = abs(weights_dict[int(wseed[1:])] - weights_dict[int(lseed[1:])])
  #temp_val = abs(weights_dict[int(wseed[1:])] - weights_dict[int(lseed[1:])])
  diff_seed.append(temp_val)
  
  diff_score.append(abs(row[1]['WScore'] - row[1]['LScore']))
  

In [31]:
CompactDF['WLProb'] = temp_win

In [32]:
CompactDF['Seed_Diff'] = diff_seed


In [33]:
CompactDF['Loc'] = loc_col

In [34]:
CompactDF['Score_Diff'] = diff_score

In [35]:
CompactDF[:10]

In [36]:
# Functionality for computing features
from pyspark.ml import feature
# Functionality for regression
from pyspark.ml import regression
# Funcionality for classification
from pyspark.ml import classification
# Object for creating sequences of transformations
from pyspark.ml import Pipeline

In [37]:

featureCols = ["DayNum", "WTeamID", "LTeamID", "Score_Diff", "Loc", "Seed_Diff", "NumOT"]
#set the input and output column names**
assembler = feature.VectorAssembler(inputCols = featureCols, outputCol = "features")
#return a dataframe with all of the  feature columns in  a vector column**


In [38]:
# Split dataset randomly into Training and Test sets. Set seed for reproducibility
import numpy as np
trainingData, validationData, testData = np.split(CompactDF, [int(0.6*len(CompactDF)), int(0.9*len(CompactDF))])

In [39]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
sql_compactDF = sqlCtx.createDataFrame(CompactDF)

sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")
sql_compactDF.show()

In [40]:
trainingData, validationData, testData = sql_compactDF.randomSplit([0.6,0.3,0.1])

In [41]:
trainingData.show()

In [42]:
from pyspark.ml.feature import StringIndexer
# Convert target into numerical categories
labelIndexer = StringIndexer(inputCol="WLProb", outputCol="label")


In [43]:
sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")
sql_compactDF.show()

In [44]:
from pyspark.ml.classification import LogisticRegression
# Train a Logistic Regression model
logisticReg = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Chain labelIndexer, vecAssembler and NBmodel in a 
pipeline = Pipeline(stages=[ assembler, logisticReg])

# Run stages in pipeline and train model
model = pipeline.fit(trainingData)

In [45]:
predictions = model.transform(validationData)

In [46]:
sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")

In [47]:
test_prediction = model.transform(testData)
test_prediction.show()

In [48]:
display(test_prediction.select("label", "prediction", "probability"))


In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction"
                                              )
accuracy = evaluator.evaluate(test_prediction)
print "Model Accuracy: ", accuracy

In [50]:
evaluator.evaluate(predictions)

In [51]:
####APPLYING RANDOM FOREST#####################

In [52]:
rdd_compactDF=sql_compactDF.rdd
rdd_compactDF.take(2)

In [53]:
CompactDF = spark.read.csv("/FileStore/tables/WNCAATourneyCompactResults.csv", header=True, inferSchema= True)

In [54]:
type(sql_compactDF)
sql_compactDF.show()

In [55]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
sql_compactDF = sqlCtx.createDataFrame(CompactDF)

sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")
#sql_compactDF.show()

In [56]:
CompactDF[:10]

In [57]:
sql_compactDF

In [58]:
Rand_DF=CompactDF
Rand_DF=Rand_DF[['Season','DayNum','WTeamID','WScore','LTeamID','LScore','NumOT','WSeed','LSeed','WLProb','Seed_Diff','Score_Diff','Loc']]
Rand_DF[:4]

In [59]:

###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ["Loc"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]#, but will run all at once later on.
  #stages += [stringIndexer, encoder]

In [60]:
cols = Rand_DF.columns
cols

In [61]:
#Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "WLProb", outputCol = "label")
stages += [label_stringIdx]

In [62]:
featureCols = ["DayNum", "WTeamID", "LTeamID", "Score_Diff", "Seed_Diff", "NumOT","Loc"]
#assemblerInputs = map(lambda c: c, categoricalColumns) + featureCols
#assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + featureCols
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
stages += [assembler]

In [63]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.


In [64]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
sql_RandDF = sqlCtx.createDataFrame(Rand_DF)
sql_RandDF.take(2)
#sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")

In [65]:
pipelineModel = pipeline.fit(sql_RandDF)
#from pyspark.sql import SQLContext
#sqlCtx = SQLContext(sc)
#sql_compactDF = sqlCtx.createDataFrame(CompactDF)

#sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")
dataset = pipelineModel.transform(sql_RandDF)
dataset.show()

In [66]:
# Keep relevant columns
#selectedcols = ["label", "features"] + cols
#dataset = dataset.select(selectedcols)
#display(dataset)

In [67]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print trainingData.count()
print testData.count()

In [68]:
trainingData.show()

In [69]:
from pyspark.sql import functions as fn
testData.select(fn.avg('label')).show()

In [70]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [71]:
Rand_DF=CompactDF
Rand_DF=Rand_DF[['Season','DayNum','WTeamID','WScore','LTeamID','LScore','NumOT','WSeed','LSeed','WLProb','Seed_Diff','Score_Diff','Loc']]
Rand_DF[:4]

In [72]:
# Make predictions on test data using the Transformer.transform() method.
predictions = rfModel.transform(testData)

In [73]:
predictions.printSchema()


In [74]:
selected = predictions.select("WLProb","label", "prediction", "probability", "WTeamID")
display(selected)

In [75]:
#We will evaluate our Random Forest model with BinaryClassificationEvaluator.

In [76]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')
evaluator.evaluate(predictions)

In [77]:
#Now we will try tuning the model with the ParamGridBuilder and the CrossValidator.

#As we indicate 3 values for maxDepth, 2 values for maxBin, and 2 values for numTrees, this grid will have 3 x 2 x 2 = 12 parameter settings for CrossValidator to choose from. We will create a 5-fold CrossValidator.

In [78]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [79]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [80]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [81]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [82]:
bestModel = cvModel.bestModel

In [83]:
# Generate predictions for entire dataset
finalPredictions = bestModel.transform(dataset)

In [84]:
# Evaluate best model
evaluator.evaluate(finalPredictions)

In [85]:
finalPredictions.createOrReplaceTempView("finalPredictions")